<a href="https://colab.research.google.com/github/s-mostafa-a/lung-cancer-segmentation/blob/master/using_vnet_for_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def _cn_t_d():
  from google.colab import drive
  drive.mount("/content/drive")

In [2]:
def _untar_data():
  !tar -xf ./drive/My\ Drive/Copy_Task06_Lung.tar -C ./

In [3]:
def connect_to_drive():
  !mkdir drive
  _cn_t_d()
  _untar_data()

In [4]:
def install_all_needed_packages():
  !pip uninstall niftidataset
  !pip install git+git://github.com/s-mostafa-a/niftidataset.git
  !pip install git+git://github.com/mattmacy/torchbiomed.git
  !git clone https://github.com/s-mostafa-a/vnet.pytorch
  !pip install hdbscan
  !mv ./vnet.pytorch ./pvn
  !git clone https://github.com/wolny/pytorch-3dunet.git
  !mv ./pytorch-3dunet ./p3dun
  !pip install SimpleITK
  !pip install setproctitle

In [5]:
def make_databunch():
  np.random.seed(1)
  train_dir = '/content/Task06_Lung/'
  tfms = tvt.Compose([ToTensor(), RandomCrop3D(110), TrimIntensity(-1000.0,1000.0,0.0,255.0), Normalize(is_3d=True)])
  tds, vds = train_val_split(source_dir=train_dir+'imagesTr', target_dir=train_dir+'labelsTr', valid_pct=0.2, transform=tfms)
  print(len(tds), len(vds))
  idb = faiv.ImageDataBunch.create(tds, vds, bs=1, num_workers=1)
  return idb

In [6]:
def make_learner(idb):
  model = VNet()
  metric = metrics.DiceCoefficient()
  metric.__name__ = 'DiceCoefficient'
  learner = faiv.Learner(idb, model, loss_func=losses.BCEDiceLoss(alpha=0.3, beta=0.7), metrics=[metric], opt_func=torch.optim.Adam)
  return learner


In [ ]:
connect_to_drive()
install_all_needed_packages()

mkdir: cannot create directory ‘drive’: File exists
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
  Cloning git://github.com/jcreinhold/niftidataset.git to /tmp/pip-req-build-2xmvceou
  Running command git clone -q git://github.com/jcreinhold/niftidataset.git /tmp/pip-req-build-2xmvceou
  Created wheel for niftidataset: filename=niftidataset-0.2.0-cp36-none-any.whl size=12356 sha256=a8bb8239838c59640f1d6fcb8a9084623c37359a19b2733fb57cda16fa6817f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-cbw306sx/

Because 3d image size are in like this: (512,512,z) which z range is from 112 to 465, I thougth it would be good if I crop each 3D image from it's third axis (z) and fix that in a constant like 110. This way raw data loss becomes so little. for example if we have an image sized (512,512, 112) we would generate an image (512,512,110), Also if we had an image sized (512, 512, 373) we would generate 3 images with sizes (512,512,110) which it covers 330 out of 373 from z of original raw data.

But when I used it with size (512,512,110), cuda went out of memory and it needed more RAM. Also I found some paper which they cropped this dataset's images to size (64,64,64)! which I thought their data loss would be so high. But anyway I went a way similar to their way and cropped original images to the size (110,110,110) which almost contains 8 times more data (and less data loss) from that paper.

So I wrote below classes which use some features of [this code](https://github.com/jcreinhold/niftidataset). It used a single image 5 times and cropped each randomly, but I thought It will not be useful because np.random.seed() makes each time just one crop. officially it wouldn't help.


In [4]:
!rm -r pvn
!rm -r pytorch-3dunet
!rm -r p3dun

rm: cannot remove 'pytorch-3dunet': No such file or directory


In [7]:
install_all_needed_packages()

Uninstalling niftidataset-0.2.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/niftidataset-0.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/niftidataset/*
Proceed (y/n)? n
  Cloning git://github.com/s-mostafa-a/niftidataset.git to /tmp/pip-req-build-mm6dshkw
  Running command git clone -q git://github.com/s-mostafa-a/niftidataset.git /tmp/pip-req-build-mm6dshkw
  Created wheel for niftidataset: filename=niftidataset-0.2.0-cp36-none-any.whl size=12420 sha256=332f349b9581a4f3412409ef167ae9fb1e25d43dc6be198ac04912c5827998c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-jguc2ei9/wheels/90/80/c0/09c0a7dbe8a2902eeca8ebc007b7cbe2b5a06389d3f71d54f9
Successfully built niftidataset
  Cloning git://github.com/mattmacy/torchbiomed.git to /tmp/pip-req-build-5gl5giv1
  Running command git clone -q git://github.com/mattmacy/torchbiomed.git /tmp/pip-req-build-5gl5giv1
  Created wheel for torchbiomed: filename=torchbiomed-0.0.1-py2.py3-none-any.whl size=10974 sha256=86bac9

In [7]:
import numpy as np
import os
import sys
module_path = os.path.abspath(os.path.join('./pvn/'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(sys.path)

['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/content/pvn']


In [8]:
from niftidataset import NiftiDataset, train_val_split, RandomCrop3D, ToTensor, Normalize, TrimIntensity
from torchvision import transforms as tvt
from torch.nn import BCEWithLogitsLoss
import torch 
from fastai import vision as faiv
%matplotlib inline
%load_ext autoreload
%autoreload 2
!export CUDA_LAUNCH_BLOCKING=1

In [9]:
from train import main
np.random.seed(1)
train_dir = '/content/Task06_Lung/'
tfms = tvt.Compose([ToTensor(), RandomCrop3D(110), TrimIntensity(-1000.0,1000.0,0.0,255.0), Normalize(is_3d=True)])
tds, vds = train_val_split(source_dir=train_dir+'imagesTr', target_dir=train_dir+'labelsTr', valid_pct=0.2, transform=tfms)
main(tds, vds)

build vnet


/content/pvn/train.py:56: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight)


  + Number of params: 45603934
loading test set
[0.       0.       0.       0.       ... 0.       0.       0.043228 0.      ]


RuntimeError: ignored